### 4.2

In [4]:
import osmium
import folium

import osmium

class TrackHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.tracks = []

    def way(self, w):
        # Check if the way is tagged as 'highway' and 'track'
        if 'highway' in w.tags and w.tags['highway'] == 'track':
            coords = []
            for pt in w.nodes:
                if pt.location.valid():  # Check if location is valid
                    coords.append((pt.lat, pt.lon))
            if coords:  # Only append if we have valid coordinates
                self.tracks.append(coords)

# Initialize the handler
handler = TrackHandler()

# Apply the handler to the OSM data file
handler.apply_file("highways-groningen.pbf")

# Create a Folium map centered on Groningen
m = folium.Map(location=[53.2, 6.6], zoom_start=12)

# Plot the tracks on the map
for track in handler.tracks:
    folium.PolyLine(track, color="blue", weight=2.5, opacity=1).add_to(m)

# Save the map
m.save("groningen_tracks_map.html")

In [5]:

class HighwayHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.highways = []

    def way(self, w):
        # Check if the way is tagged as 'highway' and is one of the desired types
        if 'highway' in w.tags and w.tags['highway'] in ['track', 'footpath', 'path']:
            coords = []
            for pt in w.nodes:
                if pt.location.valid():  # Check if the location is valid
                    coords.append((pt.lat, pt.lon))
            if coords:  # Only append if we have valid coordinates
                self.highways.append(coords)

# Initialize the handler
handler = HighwayHandler()

# Apply the handler to the OSM data file
handler.apply_file("highways-groningen.pbf")

# Create a Folium map centered on Groningen
m = folium.Map(location=[53.2, 6.6], zoom_start=12)

# Plot the highways on the map
for highway in handler.highways:
    folium.PolyLine(highway, color="blue", weight=2.5, opacity=1).add_to(m)

# Save the map
m.save("groningen_highways_map.html")

In [6]:
class HikingFootHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.routes = []

    def way(self, w):
        # Check if the way is tagged as 'highway' and is either 'hiking' or 'foot'
        if 'highway' in w.tags and w.tags['highway'] in ['hiking', 'foot']:
            coords = [(pt.lat, pt.lon) for pt in w.nodes]
            self.routes.append(coords)

# Initialize the handler
hiking_handler = HikingFootHandler()

# Apply the handler to the OSM data file
hiking_handler.apply_file("hiking-foot-groningen.pbf")

# Create a Folium map centered on Groningen
m = folium.Map(location=[53.2, 6.6], zoom_start=12)

# Plot the hiking and foot routes on the map
for route in hiking_handler.routes:
    folium.PolyLine(route, color="green", weight=2.5, opacity=1).add_to(m)

# Save the map
m.save("groningen_hiking_foot_routes_map.html")

### 4.2.3

In [7]:
import folium

# Define the coordinates
leens_coords = [53.3615233, 6.3932052]
gn_station_coords = [53.2113213, 6.5656461]

# Create a Folium map centered on the midpoint of the two points
map_center = [(leens_coords[0] + gn_station_coords[0]) / 2, (leens_coords[1] + gn_station_coords[1]) / 2]
m = folium.Map(location=map_center, zoom_start=12)

# Add markers for the start and end points
folium.Marker(location=leens_coords, popup="Leens").add_to(m)
folium.Marker(location=gn_station_coords, popup="Groningen Station").add_to(m)

# Save the map
m.save("leens_to_station_map.html")

In [8]:
import numpy as np
from sklearn.neighbors import BallTree
import networkx as nx

# Assume `nodes` is a list of tuples (latitude, longitude) for all nodes in your graph
# Example: nodes = [(lat1, lon1), (lat2, lon2), ...]

# Convert the nodes to a 2D array
nodes_array = np.array(nodes)

# Create the BallTree
tree = BallTree(nodes_array)

# Query the nearest node for the start point
start_node = tree.query(leens_coords, k=1)
end_node = tree.query(gn_station_coords, k=1)

# Get the indices of the nearest nodes
nearest_start_index = start_node[1][0][0]
nearest_end_index = end_node[1][0][0]

# Get the coordinates of the nearest nodes
nearest_start_coords = nodes_array[nearest_start_index]
nearest_end_coords = nodes_array[nearest_end_index]

print("Nearest start node coordinates:", nearest_start_coords)
print("Nearest end node coordinates:", nearest_end_coords)

NameError: name 'nodes' is not defined

In [ ]:
# Initialize the graph
G = nx.Graph()

# Example data structure for routes (should contain your actual routes)
# routes = [(node1_index, node2_index, {'length': distance}), ...]

for route in routes:
    node1_index, node2_index, length = route  # Assuming 'length' is calculated
    G.add_edge(node1_index, node2_index, weight=length)

In [ ]:
# Compute the shortest path
shortest_path = nx.shortest_path(G, source=nearest_start_index, target=nearest_end_index, weight='weight')

# Get the length of the shortest path
shortest_path_length = nx.shortest_path_length(G, source=nearest_start_index, target=nearest_end_index, weight='weight')

print("Shortest path:", shortest_path)
print("Length of the shortest path (in meters):", shortest_path_length)

In [ ]:
# Create a new Folium map for visualization
m = folium.Map(location=map_center, zoom_start=12)

# Plot the start and end points
folium.Marker(location=leens_coords, popup="Leens").add_to(m)
folium.Marker(location=gn_station_coords, popup="Groningen Station").add_to(m)

# Plot the shortest path
path_coords = [nodes[i] for i in shortest_path]
folium.PolyLine(locations=path_coords, color='green', weight=5, opacity=0.7).add_to(m)

# Save the updated map
m.save("shortest_path_map.html")

In [ ]:
print("Length of the shortest path (in meters):", shortest_path_length)